In [128]:
# Import Dependencies
import pandas as pd 
from itertools import tee, islice, chain

In [129]:
# Read in and clean up DataFrame
macro = pd.read_excel('US_Housing_Macro_Analysis.xlsx', sheet_name='Average Revenue of US Home sale')
macro = macro.iloc[3:]
macro = macro.rename(columns={'Home Sales Revenue': 'Period'})
macro.set_index('Period', inplace=True)
macro.columns = macro.iloc[0]
macro = macro.iloc[1:]
macro.index.name = None

In [130]:
# Function to access next and previous items in a for loop
# found on stack overflow: https://stackoverflow.com/questions/1011938/loop-that-also-accesses-previous-and-next-values
def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)
    

In [131]:
# Iterate over Revenue column to determine periods of Expansion and Recession
revenue = macro['Average Revenue by Year']

expansion = []
recession = []
counter1 = 0
counter2 = 0

for previous, item, nxt in previous_and_next(revenue):
    try:
        if previous < item:
            counter1 +=1
            expansion.append(counter1)
        else:
            counter1 = 0
            expansion.append(counter1)
    except:
        counter1 = 0
        expansion.append(counter1)
    try:
        if previous > item:
            counter2 +=1
            recession.append(counter2)
        else:
            counter2 = 0
            recession.append(counter2)
    except:
        counter2 = 0
        recession.append(counter2)

In [132]:
# Insert Expansion and Recession values into DF
macro.drop(columns=['Period of Expansion', 'Period of Recession'], inplace=True)
macro.insert(loc= 1, column= 'Period of Expansion', value= expansion)
macro.insert(loc= 5, column= 'Period of Recession', value= recession)

In [133]:
# Calculate percentage gained and lossed year over year
percent_change = []

for previous, item, nxt in previous_and_next(revenue):
    try:
        result = ((item - previous)/previous)
        result = "{:.2%}".format(result)
        percent_change.append(result)
    except:
        result = 0 
        result = "{:.0%}".format(result)
        percent_change.append(result)

In [134]:
# Insert percentage change into DF
macro.drop(columns= '% Change', inplace=True)
macro.insert(loc = 3, column='% Change', value = percent_change)

In [135]:
# Iterate through df to find the bottom of the recession
recession = macro['Period of Recession']
valley = []
for previous, item, nxt in previous_and_next(recession):
    try:
        if nxt < item > previous:
            valley.append('Yes')
        else:
            valley.append('No')
    except:
        valley.append('No')

In [136]:
# Iterate through df to find the peak of the expansion
expansion = macro['Period of Expansion']
peak = []
for previous, item, nxt in previous_and_next(expansion):
    try:
        if nxt < item > previous:
            peak.append('Yes')
        else:
            peak.append('No')
    except:
        peak.append('No')

In [137]:
# Insert peaks and valleys into DF
macro.drop(columns= ['Peak', 'Valley'], inplace=True)
macro.insert(loc= 2, column='Peak', value = peak)
macro.insert(loc= 4, column='Valley', value= valley)

In [138]:
# Format the output
macro['Average Revenue by Year'] = (macro['Average Revenue by Year'] / 1000000000)
pd.options.display.float_format = "{:,.2f}".format

In [139]:
macro

Period,Average Revenue by Year,Period of Expansion,Peak,% Change,Valley,Period of Recession
1963,10.81,0,No,0%,No,0
1964,11.58,1,No,7.17%,No,0
1965,12.36,2,Yes,6.73%,No,0
1966,10.74,0,No,-13.11%,Yes,1
1967,11.98,1,No,11.53%,No,0
1968,13.03,2,Yes,8.80%,No,0
1969,12.50,0,No,-4.10%,Yes,1
1970,12.90,1,No,3.21%,No,0
1971,18.56,2,No,43.90%,No,0
1972,21.90,3,No,17.96%,No,0


In [140]:
# Slice dataframe into 10 year intervals
macro_1 = macro[0:10]
macro_2 = macro[10:20]
macro_3 = macro[20:30]
macro_4 = macro[30:40]
macro_5 = macro[40:50]
macro_6 = macro[50:59]
macro_6

Period,Average Revenue by Year,Period of Expansion,Peak,% Change,Valley,Period of Recession
2013,139.21,2,No,29.46%,No,0
2014,151.94,3,No,9.15%,No,0
2015,176.70,4,No,16.29%,No,0
2016,202.46,5,No,14.58%,No,0
2017,235.94,6,No,16.54%,No,0
2018,237.54,7,No,0.68%,No,0
2019,262.20,8,No,10.38%,No,0
2020,322.14,9,No,22.86%,No,0
2021,357.90,10,No,11.10%,No,0


In [141]:
# Export to excel workbook
#with pd.ExcelWriter('US Housing Annual Revenue.xlsx') as writer:  
    macro.to_excel(writer, sheet_name='Annual Revenue', index_label='Period')
    macro_1.to_excel(writer, sheet_name="63-72", index_label='Period')
    macro_2.to_excel(writer, sheet_name="73-82", index_label='Period')
    macro_3.to_excel(writer, sheet_name="83-92", index_label='Period')
    macro_4.to_excel(writer, sheet_name="93-02", index_label='Period')
    macro_5.to_excel(writer, sheet_name="03-12", index_label='Period')
    macro_6.to_excel(writer, sheet_name="13-21", index_label='Period')
